## Import items

In [16]:
!ls /content/drive/MyDrive/BCG/case_studies_anlysis/scripts/

driver.py  spark_submit.sh


In [23]:
!spark-submit --deploy-mode client --py-files /content/drive/MyDrive/BCG/case_studies_anlysis/dist/crashanalysis-0.0.1-py3-none-any.whl /content/drive/MyDrive/BCG/case_studies_anlysis/scripts/driver.py content/drive/MyDrive/BCG/case_studies_anlysis/

24/08/25 11:28:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
******system arguments********
['/content/drive/MyDrive/BCG/case_studies_anlysis/scripts/driver.py', 'content/drive/MyDrive/BCG/case_studies_anlysis/']
******system arguments********
24/08/25 11:28:24 INFO SparkContext: Running Spark version 3.5.2
24/08/25 11:28:24 INFO SparkContext: OS info Linux, 6.1.85+, amd64
24/08/25 11:28:24 INFO SparkContext: Java version 11.0.24
24/08/25 11:28:24 INFO ResourceUtils: ==============================================================
24/08/25 11:28:24 INFO ResourceUtils: No custom resources configured for spark.driver.
24/08/25 11:28:24 INFO ResourceUtils: ==============================================================
24/08/25 11:28:24 INFO SparkContext: Submitted application: SuppressWarning
24/08/25 11:28:24 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, 

In [5]:
pip install pyspark


In [ ]:
# Import
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com') \
                    .getOrCreate()

In [ ]:
!ls /content/drive/MyDrive/BCG/Dastasets/Data/

/Users/ssm7/IdeaProjects/case_studies_anlysis/configs/driver.json

Charges_use.csv  Endorse_use.csv	 Restrict_use.csv
Damages_use.csv  Primary_Person_use.csv  Units_use.csv


In [ ]:
from pyspark.sql.functions import col,trim,count,like,upper,lower

## Basic config input setup

In [ ]:
base_root='/content/drive/MyDrive/BCG/Dastasets/Data/'
datasets_config={
"Charges":"Charges_use.csv",
"Endorsements":"Endorse_use.csv",
"Restrict":"Restrict_use.csv",
"PrimaryPerson":"Primary_Person_use.csv",
"Unit":"Units_use.csv",
"Damages":"Damages_use.csv",
}

In [ ]:
def load_dataframe(key):
  return spark.read.csv(base_root+datasets_config[key],header=True,inferSchema=True)


## Analytics 1: Find the number of crashes (accidents) in which number of males killed are greater than 2?

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, count, sum
from typeguard import typechecked  # Import typeguard for runtime type checking

def get_crashes_with_high_deaths(males_killed_per_crash: int):
    print('get_crashes_with_high_deaths')
    """
    Returns a DataFrame containing crash IDs with a count of deaths greater than males_killed_per_crash.

    Args:
        males_killed_per_crash (int): The minimum number of males killed per crash.

    Returns:
        DataFrame: A DataFrame containing crash IDs with a count of deaths greater than males_killed_per_crash.
    """
    data = [Row(no_of_crashes='not_able_to_convert')]
    try:
        if not(isinstance(males_killed_per_crash, int)):
            males_killed_per_crash=int(males_killed_per_crash)

        no_of_crashes_men_killed = load_dataframe('PrimaryPerson') \
            .select('crash_id', 'PRSN_GNDR_ID', 'DEATH_CNT') \
            .filter((trim(col('PRSN_GNDR_ID')) == 'MALE') & (col('DEATH_CNT') >= 1)) \
            .groupBy('crash_id') \
            .agg(sum(col('DEATH_CNT')).alias('count_of_deaths')) \
            .filter(col('count_of_deaths') > males_killed_per_crash) \
            .select('crash_id') \
            .distinct().count()

        data = [Row(no_of_crashes=no_of_crashes_men_killed)]

    except (ValueError) as e:
        print(f"Type error occurred , not able to convert the input to int type check your input : {str(e)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        return spark.createDataFrame(data)




get_crashes_with_high_deaths('2').show()


get_crashes_with_high_deaths
+-------------+
|no_of_crashes|
+-------------+
|            0|
+-------------+



In [ ]:
# Analytics 1: Find the number of crashes (accidents) in which number of males killed are greater than 2?

load_dataframe('PrimaryPerson').select('crash_id').distinct().count()
# 83805
print(load_dataframe('PrimaryPerson').count())

156954


In [ ]:
#duplicate crash id's
load_dataframe('PrimaryPerson').select('crash_id').groupBy('crash_id').count().orderBy('count',ascending=False).show()

+--------+-----+
|crash_id|count|
+--------+-----+
|15146770|   14|
|14918153|   12|
|15487600|   12|
|15460374|   10|
|15329323|   10|
|14987833|    9|
|15097577|    8|
|15086342|    8|
|14878529|    8|
|15418533|    8|
|15192049|    8|
|15364029|    8|
|14914046|    7|
|14976637|    7|
|15292692|    7|
|15121746|    7|
|15032281|    7|
|14964331|    7|
|15016597|    7|
|15219679|    7|
+--------+-----+
only showing top 20 rows



In [ ]:
load_dataframe('PrimaryPerson').filter(col('crash_id')=='15146770').show()

+--------+--------+--------+------------+------------------+--------------------+--------+-----------------+------------+------------+-------------------+---------------+--------------+-----------+---------------------+----------------+------------------+---------------------+----------------+-----------------+---------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+----------------+-----------------+---------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|PRSN_TYPE_ID|PRSN_OCCPNT_POS_ID|   PRSN_INJRY_SEV_ID|PRSN_AGE|PRSN_ETHNICITY_ID|PRSN_GNDR_ID|PRSN_EJCT_ID|       PRSN_REST_ID| PRSN_AIRBAG_ID|PRSN_HELMET_ID|PRSN_SOL_FL|PRSN_ALC_SPEC_TYPE_ID|PRSN_ALC_RSLT_ID|PRSN_BAC_TEST_RSLT|PRSN_DRG_SPEC_TYPE_ID|PRSN_DRG_RSLT_ID|DRVR_DRG_CAT_1_ID|PRSN_DEATH_TIME|INCAP_INJRY_CNT|NONINCAP_INJRY_CNT|POSS_INJRY_CNT|NON_INJRY_CNT|UNKN_INJRY_CNT|TOT_INJRY_CNT|DEATH_CNT|DRVR_LIC_TYPE_ID|DRVR_LIC_STATE_ID|DRVR_LIC_CLS_ID|DRVR_ZIP|
+--------+------

In [ ]:
load_dataframe('PrimaryPerson').select('DEATH_CNT').filter.distinct().show()

AttributeError: 'function' object has no attribute 'distinct'

In [ ]:
# Analytics 1: Find the number of crashes (accidents) in which number of males killed are greater than 2?
load_dataframe('PrimaryPerson') \
.select('crash_id','PRSN_GNDR_ID','DEATH_CNT') \
.filter( (trim(col('PRSN_GNDR_ID'))=='MALE') & (col('DEATH_CNT')>=1)  ) \
.groupBy('crash_id') \
.agg(count(col('DEATH_CNT')).alias('count_of_deaths')) \
.filter(col('count_of_deaths')>2) \
.count()

In [ ]:
load_dataframe('Unit').printSchema()

In [ ]:
# Analytics 1: Find the number of crashes (accidents) in which number of males killed are greater than 2?
load_dataframe('Unit') \
.select('DEATH_CNT','crash_id') \
.filter( (col('DEATH_CNT')>=1)  ) \
.groupBy('crash_id') \
.agg(count(col('DEATH_CNT')).alias('count_of_deaths')) \
.orderBy('count_of_deaths',ascending=False)\
.show(truncate=False)



In [ ]:
load_dataframe('PrimaryPerson')\
.filter(trim(col('crash_id'))=='15146770')\
.show(truncate=False)
# 15379024
# 15429998


In [ ]:
load_dataframe('Unit') \
.filter(trim(col('crash_id'))=='15072267') \
.show(truncate=False)

In [ ]:
load_dataframe('PrimaryPerson') \
.select('crash_id','PRSN_GNDR_ID','DEATH_CNT') \
.filter(  (col('DEATH_CNT')==1) & (col('DEATH_CNT')>=1) ) \
.groupBy('crash_id') \
.agg(count(col('DEATH_CNT')).alias('count_of_deaths')) \
.filter(col('count_of_deaths')>=2).show(truncate=False)

 ## Analysis 2: How many two wheelers are booked for crashes?


In [ ]:
def count_two_wheelers():
    """
    Counts the number of two wheelers booked for crashes.

    Returns:
        int: The count of two wheelers.
    """
    data = [Row(two_wheelers_count='not_able_to_convert')]
    try:
        df = load_dataframe('Unit')
        count_two_wheelers = df.filter(upper(col('VEH_BODY_STYL_ID')).rlike('MOTORCYCLE')).count()
        data = [Row(two_wheelers_count=count_two_wheelers)]
    except Exception as e:
        print(f"An error occurred: {str(e)}")

    finally:
        return spark.createDataFrame(data)
count_two_wheelers().show()

+------------------+
|two_wheelers_count|
+------------------+
|               784|
+------------------+



In [ ]:
load_dataframe('Unit') \
.select('VEH_BODY_STYL_ID') \
.distinct().show(truncate=False)

+---------------------------------+
|VEH_BODY_STYL_ID                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|POLICE MOTORCYCLE                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|NOT REPORTED                     |
+---------------------------------+



In [ ]:
# Analysis 2: How many two wheelers are booked for crashes?

load_dataframe('Unit') \
.filter(upper(col('VEH_BODY_STYL_ID')).rlike('MOTORCYCLE')) \
.count()


784

## Analysis 3: Determine the Top 5 Vehicle Makes of the cars present in the crashes in which driver died and Airbags did not deploy.


In [ ]:
def get_top_vehicle_makes(n_filter):

    """
    Get the top n_filter vehicle makes of the cars present in the crashes in which driver died and Airbags did not deploy.

    Args:
        n_filter (int): Number of top vehicle makes to retrieve.

    Returns:
        DataFrame: DataFrame containing the top n_filter vehicle makes.
    """
    data = [Row(top_n_makes='some issue with the parmeter')]
    top_n_vechile_makes_death_abg_notdeployed_df=spark.createDataFrame(data)
    try:
        # Filtering person as a driver died in the accident because airbag didn't deploy
        df_airbag_notdeployed = load_dataframe('PrimaryPerson') \
            .filter((trim(col('PRSN_TYPE_ID')) == 'DRIVER') & (col('DEATH_CNT') >= 1) & (trim(col('PRSN_AIRBAG_ID')).rlike('NOT\\s*DEPLOYED'))) \
            .select(['crash_id', 'UNIT_NBR', 'PRSN_AIRBAG_ID', 'DEATH_CNT'])

        # Filtering vehicles only cars and their make details
        df_vechiledesignfilter = load_dataframe('Unit') \
            .filter(trim(col('VEH_BODY_STYL_ID')).rlike('PASSENGER\\s*CAR')) \
            .select(['crash_id', 'UNIT_NBR', 'VEH_BODY_STYL_ID', 'VEH_MAKE_ID'])

        # Joining airbag deployment details with the vehicle details and finding top n_filter makes causing more deaths
        top_n_vechile_makes_death_abg_notdeployed_df = df_airbag_notdeployed.join(df_vechiledesignfilter, ['crash_id', 'UNIT_NBR'], 'inner') \
            .groupBy('VEH_MAKE_ID') \
            .agg(count(col('*')).alias('occurence_of_the_make')) \
            .withColumn('make_rank', row_number().over(Window.orderBy(desc('occurence_of_the_make')))) \
            .filter(col('make_rank') <= n_filter) \
            .select('VEH_MAKE_ID').alias('top_n_vechile_makes')


    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        return top_n_vechile_makes_death_abg_notdeployed_df



get_top_vehicle_makes(5).show()

+-----------+
|VEH_MAKE_ID|
+-----------+
|     NISSAN|
|  CHEVROLET|
|       FORD|
|      HONDA|
|    PONTIAC|
+-----------+



In [ ]:
#Analysis 3: Determine the Top 5 Vehicle Makes of the cars present in the crashes in which driver died and Airbags did not deploy.
from pyspark.sql.functions import rank,row_number,desc
from pyspark.sql.window import Window

#filtering person as a driver died in the accident because of airbag didn't deploy
df_airbag_notdeployed=load_dataframe('PrimaryPerson') \
.filter((trim(col('PRSN_TYPE_ID'))=='DRIVER') & (col('DEATH_CNT')>=1) & (trim(col('PRSN_AIRBAG_ID')).rlike('NOT\\s*DEPLOYED') ) ) \
.select(['crash_id','UNIT_NBR','PRSN_AIRBAG_ID','DEATH_CNT'])


#filtering vehicles only cars and their make details
df_vechiledesignfilter=load_dataframe('Unit') \
.filter(trim(col('VEH_BODY_STYL_ID')).rlike('PASSENGER\\s*CAR') ) \
.select(['crash_id','UNIT_NBR','VEH_BODY_STYL_ID','VEH_MAKE_ID'])


top_n_filter_value=5

#joining airbag deployment details with the vehicle details and finding top 5 makes causes more deaths
top_n_vechile_makes_death_abg_notdeployed_df=df_airbag_notdeployed.join(df_vechiledesignfilter,['crash_id','UNIT_NBR'],'inner')\
.groupBy('VEH_MAKE_ID')\
.agg(count(col('*')).alias('occurence_of_the_make'))\
.withColumn('make_rank',row_number().over(Window.orderBy(desc('occurence_of_the_make'))))\
.filter(col('make_rank')<=top_n_filter_value)\
.select('VEH_MAKE_ID')


# +-----------+---------------------+
# |VEH_MAKE_ID|occurence_of_the_make|
# +-----------+---------------------+
# |NISSAN     |4                    |
# |CHEVROLET  |3                    |
# |FORD       |2                    |
# |HONDA      |2                    |
# |PONTIAC    |1                    |
# +-----------+---------------------+

+-----------+
|VEH_MAKE_ID|
+-----------+
|NISSAN     |
|CHEVROLET  |
|FORD       |
|HONDA      |
|PONTIAC    |
|CADILLAC   |
+-----------+



In [ ]:
df_airbag_notdeployed=load_dataframe('PrimaryPerson')\
.filter( (col('DEATH_CNT')>=1) & (trim(col('PRSN_AIRBAG_ID')).rlike('NOT\\s*DEPLOYED') ) )\
.filter(col('CRASH_ID'))\
.select(['crash_id','UNIT_NBR','PRSN_AIRBAG_ID','DEATH_CNT'])
df_airbag_notdeployed.show(5,truncate=False)

+--------+--------+--------------+---------+
|crash_id|UNIT_NBR|PRSN_AIRBAG_ID|DEATH_CNT|
+--------+--------+--------------+---------+
|14848300|2       |NOT DEPLOYED  |1        |
+--------+--------+--------------+---------+



In [ ]:
load_dataframe('Unit') \
.select(col('VEH_BODY_STYL_ID')).distinct().show(truncate=False)

+---------------------------------+
|VEH_BODY_STYL_ID                 |
+---------------------------------+
|BUS                              |
|NA                               |
|VAN                              |
|PICKUP                           |
|SPORT UTILITY VEHICLE            |
|PASSENGER CAR, 4-DOOR            |
|FIRE TRUCK                       |
|TRUCK                            |
|UNKNOWN                          |
|AMBULANCE                        |
|POLICE CAR/TRUCK                 |
|MOTORCYCLE                       |
|YELLOW SCHOOL BUS                |
|POLICE MOTORCYCLE                |
|PASSENGER CAR, 2-DOOR            |
|TRUCK TRACTOR                    |
|FARM EQUIPMENT                   |
|NEV-NEIGHBORHOOD ELECTRIC VEHICLE|
|OTHER  (EXPLAIN IN NARRATIVE)    |
|NOT REPORTED                     |
+---------------------------------+



In [ ]:
df_vechiledesignfilter=load_dataframe('Unit') \
.filter(trim(col('VEH_BODY_STYL_ID')).rlike('PASSENGER\\s*CAR') ) \
.filter(col('CRASH_ID')=='14848300')\
.select(['crash_id','UNIT_NBR','VEH_BODY_STYL_ID','DEATH_CNT'])
df_vechiledesignfilter.show(5,truncate=False)

+--------+--------+---------------------+---------+
|crash_id|UNIT_NBR|VEH_BODY_STYL_ID     |DEATH_CNT|
+--------+--------+---------------------+---------+
|14848300|2       |PASSENGER CAR, 4-DOOR|1        |
+--------+--------+---------------------+---------+



In [ ]:
df_airbag_notdeployed.join(df_vechiledesignfilter,['crash_id','UNIT_NBR'],'left').show(5,truncate=False)

+--------+--------+--------------+---------+---------------------+---------+
|crash_id|UNIT_NBR|PRSN_AIRBAG_ID|DEATH_CNT|VEH_BODY_STYL_ID     |DEATH_CNT|
+--------+--------+--------------+---------+---------------------+---------+
|14848300|2       |NOT DEPLOYED  |1        |PASSENGER CAR, 4-DOOR|1        |
+--------+--------+--------------+---------+---------------------+---------+



In [ ]:
#Analysis 3: Determine the Top 5 Vehicle Makes of the cars present in the crashes in which driver died and Airbags did not deploy.
load_dataframe('PrimaryPerson')\
.select('crash_id','DEATH_CNT','PRSN_AIRBAG_ID')\
.filter( (col('DEATH_CNT')>=1) & (trim(col('PRSN_AIRBAG_ID')).rlike('NOT\\s*DEPLOYED') ) )\
.count()

# rlike('NOT\\s*DEPLOYED'):


33

In [ ]:
#Analysis 3: Determine the Top 5 Vehicle Makes of the cars present in the crashes in which driver died and Airbags did not deploy.
load_dataframe('PrimaryPerson')\
.select('crash_id','DEATH_CNT','PRSN_AIRBAG_ID')\
.filter( (col('DEATH_CNT')>=1) )\
.select('PRSN_AIRBAG_ID')\
.distinct()\
.show(truncate=False)

+-----------------+
|PRSN_AIRBAG_ID   |
+-----------------+
|NOT DEPLOYED     |
|DEPLOYED, SIDE   |
|UNKNOWN          |
|NOT APPLICABLE   |
|DEPLOYED MULTIPLE|
|DEPLOYED, FRONT  |
+-----------------+



In [ ]:
#Analysis 3: Determine the Top 5 Vehicle Makes of the cars present in the crashes in
# which driver died and Airbags did not deploy.

#filtering person as a driver died in the accident becuase of airbag didnt deploy
df_airbag_notdeployed=load_dataframe('PrimaryPerson') \
.filter((trim(col('PRSN_TYPE_ID'))=='DRIVER') & (col('DEATH_CNT')>=1) & (trim(col('PRSN_AIRBAG_ID')).rlike('NOT\\s*DEPLOYED') ) ) \
.select(['crash_id','UNIT_NBR','PRSN_AIRBAG_ID','DEATH_CNT'])

#filtering vechiles only cars and their make details
df_vechiledesignfilter=load_dataframe('Unit') \
.filter(trim(col('VEH_BODY_STYL_ID')).rlike('PASSENGER\\s*CAR') ) \
.select(['crash_id','UNIT_NBR','VEH_BODY_STYL_ID','VEH_MAKE_ID'])

# print(df_airbag_notdeployed.join(df_vechiledesignfilter,['crash_id','UNIT_NBR'],'inner').count())
# df_airbag_notdeployed.join(df_vechiledesignfilter,['crash_id','UNIT_NBR'],'inner').show(30,truncate=False)
df_airbag_notdeployed.printSchema()
df_vechiledesignfilter.printSchema()

#joining airbag deployment details with the vechile details and finding top 5 makes causes more deaths
df_airbag_notdeployed.join(df_vechiledesignfilter,['crash_id','UNIT_NBR'],'inner')\
.groupBy('VEH_MAKE_ID')\
.agg(count(col('*')).alias('occurence_of_the_make'))\
.orderBy('occurence_of_the_make',ascending=False)\
.limit(5)\
.show(truncate=False)

root
 |-- crash_id: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- PRSN_AIRBAG_ID: string (nullable = true)
 |-- DEATH_CNT: integer (nullable = true)

root
 |-- crash_id: integer (nullable = true)
 |-- UNIT_NBR: integer (nullable = true)
 |-- VEH_BODY_STYL_ID: string (nullable = true)
 |-- VEH_MAKE_ID: string (nullable = true)

+-----------+---------------------+
|VEH_MAKE_ID|occurence_of_the_make|
+-----------+---------------------+
|NISSAN     |4                    |
|CHEVROLET  |3                    |
|FORD       |2                    |
|HONDA      |2                    |
|PONTIAC    |1                    |
+-----------+---------------------+



## Analysis 4: Determine number of Vehicles with driver having valid licences involved in hit and run?

In [ ]:

#Analysis 4: Determine number of Vehicles with driver having valid
#Licenses involved in hit and run?
# .filter(~col('VIN').isin(UNLICENSED_pattern))

def count_hit_and_run_vehicles():
    """
    This function determines the number of vehicles with drivers having valid licenses involved in hit and run.

    Returns:
        int: The number of hit and run vehicles with valid drivers' licenses.
    """
    try:
        data = [Row(count_hit_and_run_vehicles='not_able_to_convert')]
        # Filtering all the negative patterns which need to be filtered negative boolean
        UNLICENSED_pattern = ['UNLICENSED', 'UNKNOWN', 'OTHER', 'NA', 'NULL']

        # Filtering records with driver having valid license
        vaid_driverlicense_crashes = load_dataframe('PrimaryPerson') \
            .filter(upper(col('PRSN_TYPE_ID')) == 'DRIVER') \
            .select('DRVR_LIC_TYPE_ID', 'CRASH_ID', 'UNIT_NBR') \
            .filter(~col('DRVR_LIC_TYPE_ID').isin(UNLICENSED_pattern))

        # Pulling the records of the hit and run vehicle details
        intermed_vechile_lisc_details = vaid_driverlicense_crashes.join(
            load_dataframe('Unit').select(['CRASH_ID', 'UNIT_NBR', 'VEH_HNR_FL', 'VIN'])
            .filter(col('VEH_HNR_FL') == 'Y'),
            ['CRASH_ID', 'UNIT_NBR'], 'inner')

        # Return the count of hit and run vehicles with valid drivers' licenses
        count_hit_and_run_vehicles_count= intermed_vechile_lisc_details.count()
        data = [Row(count_hit_and_run_vehicles=count_hit_and_run_vehicles_count)]

    except Exception as e:
        # Handle any exceptions that occur during the execution of the function
        print(f"An error occurred: {str(e)}")
    finally:
        return spark.createDataFrame(data)
count_hit_and_run_vehicles().show()

+--------------------------+
|count_hit_and_run_vehicles|
+--------------------------+
|                      3118|
+--------------------------+



In [ ]:
load_dataframe('Unit').select('VEH_HNR_FL').groupBy('VEH_HNR_FL').count().show(truncate=False)

+----------+------+
|VEH_HNR_FL|count |
+----------+------+
|NULL      |5736  |
|Y         |4971  |
|N         |162792|
+----------+------+



In [ ]:
load_dataframe('Unit').select('VEH_HNR_FL').filter(col('VEH_HNR_FL')=='Y').count()

4971

In [ ]:
from pyspark.sql.functions import desc
load_dataframe('Unit').filter(col('VEH_HNR_FL')=='Y').filter(~col('VIN').isin(UNLICENSED_pattern))\
.select('VIN').groupBy('VIN').count().orderBy(desc("count")).show()

+-----------------+-----+
|              VIN|count|
+-----------------+-----+
|1FTRW07L23KA57377|    6|
|1FTRX17W3XKB38908|    5|
|              UNK|    5|
|1XKDDB9XX1R873693|    4|
|1FTRX17L1XKB99454|    3|
|1GTEK14K8NZ515381|    3|
|1D7HA18N78S580746|    3|
|1G1BN52P4RR107166|    3|
|1C6RR6GT2FS691777|    2|
|1FDXF46P17EA98697|    2|
|1J4G248S2YC373423|    2|
|3VWFE21CX2M442149|    2|
|                0|    2|
|1GNEC13Z53R250693|    2|
|1G8AG52F35Z106258|    2|
|4T1BE32K25U579370|    2|
|1XP5DB9X84D832284|    2|
|1D7HA18216S684779|    2|
|1GCHC53K99F110625|    2|
|1FUJGLCK19LAE8804|    2|
+-----------------+-----+
only showing top 20 rows



In [ ]:
load_dataframe('Restrict').select('DRVR_LIC_RESTRIC_ID').distinct().show(50,truncate=False)

+-----------------------------------------------------------------+
|DRVR_LIC_RESTRIC_ID                                              |
+-----------------------------------------------------------------+
|MOPED (EFF. END 12/31/2016)                                      |
|APPLICABLE PROSTHETIC DEVICES                                    |
|FRSI CDL MM/DD/YY - MM/DD/YY OR EXEMPT A VEH                     |
|TRC 545.424 APPLIES UNTIL MM/DD/YY                               |
|TO/FROM WORK OR LOFS 21 OR OVER                                  |
|INTRASTATE ONLY (EFF. 1/1/2017)                                  |
|WITH CORRECTIVE LENSES                                           |
|STATED ON LICENSE                                                |
|OPERATION CLASS A EXEMPT VEH AUTHORIZED                          |
|VEHICLE NOT TO EXCEED CLASS C (EFF. END 12/31/2016)              |
|NO CLASS A AND B PASSENGER VEHICLE (EFF. 1/1/2017)               |
|IF CMV, CUSTOM-HARVESTING INTERSTATE           

In [ ]:
load_dataframe('Endorsements').select('DRVR_LIC_ENDORS_ID').distinct().show(50,truncate=False)

+-------------------------------------+
|DRVR_LIC_ENDORS_ID                   |
+-------------------------------------+
|SCHOOL BUS                           |
|OTHER/OUT OF STATE                   |
|UNKNOWN                              |
|HAZARDOUS MATERIALS                  |
|UNLICENSED                           |
|TANK VEHICLE                         |
|NONE                                 |
|TANK VEHICLE WITH HAZARDOUS MATERIALS|
|PASSENGER                            |
|DOUBLE/TRIPLE TRAILER                |
+-------------------------------------+



In [ ]:
load_dataframe('Charges')\
.filter(lower(col('CHARGE')).rlike('hit.*run')) \
.show(50,truncate=False)


+--------+--------+--------+---------------------------------------------------+-------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|CHARGE                                             |CITATION_NBR |
+--------+--------+--------+---------------------------------------------------+-------------+
|14873477|1       |1       |HIT AND RUN $200                                   |NULL         |
|14905247|1       |1       |DWI; HIT AND RUN                                   |16-12381     |
|14905736|1       |1       |HIT & RUN                                          |4045868-01   |
|14911483|1       |1       |HIT & RUN                                          |4045939      |
|14925033|1       |1       |HIT AND RUN                                        |16-0041465   |
|14928814|1       |1       |ACCIDENT HIT AND RUN                               |PENDING      |
|14944845|1       |1       |HIT AND RUN MORE THAN $200                         |160224028    |
|14947443|1       |1       |ACCIDENT HIT AND RUN  

In [ ]:
#Analysis 4: Determine number of Vehicles with driver having valid
#licences involved in hit and run?
load_dataframe('Unit').filter(col('VEH_HNR_FL')=='Y').show(50,truncate=False)

+--------+--------+-------------+-------------+----------+----------------+-----------------+------------+------------+-----------+-------------------+---------------------+---------------+--------+-----------------+----------------------------+---------------------------------------------+-----------------+--------------+--------------------------------------------+-----------------+--------------+------------------+-----------------------------------+----------------------------------------+----------------------------------+------------------------------+------------------------------+---------------+---------------------+---------------+------------------+--------------+-------------+--------------+-------------+---------+
|CRASH_ID|UNIT_NBR|UNIT_DESC_ID |VEH_PARKED_FL|VEH_HNR_FL|VEH_LIC_STATE_ID|VIN              |VEH_MOD_YEAR|VEH_COLOR_ID|VEH_MAKE_ID|VEH_MOD_ID         |VEH_BODY_STYL_ID     |EMER_RESPNDR_FL|OWNR_ZIP|FIN_RESP_PROOF_ID|FIN_RESP_TYPE_ID            |VEH_DMAG_AREA_1_ID

In [ ]:
#Analysis 4: Determine number of Vehicles with driver having valid
#licences involved in hit and run?
# .filter(~col('VIN').isin(UNLICENSED_pattern))
from pyspark.sql.functions import upper

#filtering all the negatvie patterns which need to be filtered negative boolean
UNLICENSED_pattern= ['UNLICENSED','UNKNOWN','OTHER','NA','NULL']

#filtering records with driver having valid liscence
vaid_driverlicense_crashes=load_dataframe('PrimaryPerson')\
.filter(upper(col('PRSN_TYPE_ID'))=='DRIVER')\
.select('DRVR_LIC_TYPE_ID','CRASH_ID','UNIT_NBR')\
.filter(~col('DRVR_LIC_TYPE_ID').isin(UNLICENSED_pattern))

vaid_driverlicense_crashes.show(5,truncate=False)

#pulling the records of the hit and run vechile details
intermed_vechile_lisc_details=vaid_driverlicense_crashes.join(load_dataframe('Unit').select(['CRASH_ID','UNIT_NBR','VEH_HNR_FL','VIN'])\
                                .filter(col('VEH_HNR_FL')=='Y')\
                                ,['CRASH_ID','UNIT_NBR'],'inner')
# intermed_vechile_lisc_details.show(5,truncate=False)
print(intermed_vechile_lisc_details.count())

print(intermed_vechile_lisc_details.select('VIN').distinct().count())
# 3127
# 3023

+----------------------+--------+--------+
|DRVR_LIC_TYPE_ID      |CRASH_ID|UNIT_NBR|
+----------------------+--------+--------+
|DRIVER LICENSE        |14768622|1       |
|DRIVER LICENSE        |14838637|1       |
|COMMERCIAL DRIVER LIC.|14838637|2       |
|DRIVER LICENSE        |14838641|1       |
|ID CARD               |14838641|2       |
+----------------------+--------+--------+
only showing top 5 rows

3118
3014


## Analysis 5: Which state has highest number of accidents in which females are not involved?

In [ ]:
#Analysis 5: Which state has highest number of
#accidents in which females are not involved?
from pyspark.sql import  DataFrame

def get_top_states_with_highest_accidents(top_N_states)-> DataFrame:
    """
    Returns a dataframe with the top N states that have the highest number of accidents
    in which females are not involved.

    Args:
        top_N_states (int): The number of top states to return.

    Returns:
        DataFrame: A dataframe with the top N states.
    """
    try:
        null_pattern = ['UNLICENSED', 'UNKNOWN', 'OTHER', 'NA', 'NULL']

        df= load_dataframe('PrimaryPerson') \
            .filter(~upper(col('DRVR_LIC_STATE_ID')).isin(null_pattern)) \
            .filter((upper(col('PRSN_GNDR_ID')) != 'FEMALE') | (upper(col('PRSN_GNDR_ID')) != 'NA')) \
            .groupBy('DRVR_LIC_STATE_ID') \
            .agg(count(col('DRVR_LIC_STATE_ID')).alias('count_state_wise')) \
            .withColumn('state_rank', rank().over(Window.orderBy(desc('count_state_wise')))) \
            .filter(col('state_rank') <= top_N_states) \
            .withColumn('top_state_in_acc_otherthanfemale', trim(col('DRVR_LIC_STATE_ID'))) \
            .select('top_state_in_acc_otherthanfemale')
    except Exception as e:
      print(f"An key error occurred check the schema of dataframe: {str(e)}")
      df=spark.createDataFrame([(str(e),)], ["issues_presists"])
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        df=spark.createDataFrame([(None,)], ["issues_presists"])
    finally:
        return df
get_top_states_with_highest_accidents(1).show()




+--------------------------------+
|top_state_in_acc_otherthanfemale|
+--------------------------------+
|                           Texas|
+--------------------------------+



In [ ]:
#Analysis 5: Which state has highest number of
#accidents in which females are not involved?
null_pattern= ['UNLICENSED','UNKNOWN','OTHER','NA','NULL']


load_dataframe('PrimaryPerson')\
.filter(~upper(col('DRVR_LIC_STATE_ID')).isin(null_pattern))\
.filter((upper(col('PRSN_GNDR_ID'))!='FEMALE') | (upper(col('PRSN_GNDR_ID'))!='NA') )\
.groupBy('DRVR_LIC_STATE_ID')\
.agg(count(col('DRVR_LIC_STATE_ID')).alias('count_state_wise'))\
.orderBy('count_state_wise',ascending=False)\
.limit(1)\
.show(truncate=False)

+-----------------+----------------+
|DRVR_LIC_STATE_ID|count_state_wise|
+-----------------+----------------+
|Texas            |136335          |
+-----------------+----------------+



## Analysis 6: Which are the Top 3rd to 5th VEH_MAKE_IDs that contribute to a largest number of injuries including death

In [ ]:
def get_top_vehicle_injuries(limit_start: int, limit_end: int) -> DataFrame:
    """
    Retrieves the top vehicle makes that contribute to the largest number of injuries including death.

    Args:
        limit_start (int): The starting ordinality to consider.
        limit_end (int): The ending ordinality to consider.

    Returns:
        DataFrame: A DataFrame containing the top vehicle makes.

    Raises:
        ValueError: If the limits are not valid or if the DataFrame fails to load.
        Exception: If any other error occurs during the execution.
    """
    try:
        if not isinstance(limit_start, int) or not isinstance(limit_end, int):
            limit_start=int(limit_start)
            limit_end=int(limit_end)


        df_vehicle_wise_injury_death_count = load_dataframe('Unit').select(['TOT_INJRY_CNT', 'DEATH_CNT', 'VEH_MAKE_ID']) \
            .withColumn('total_injury_death_count', trim(col('TOT_INJRY_CNT')) + trim(col('DEATH_CNT'))) \
            .groupBy('VEH_MAKE_ID') \
            .agg(sum(col('total_injury_death_count')).alias('vech_wise_total_injury_death_count'))

        df_vehicle_wise_injury_death_count = df_vehicle_wise_injury_death_count \
            .withColumn("ordinality_injury_death_count", rank().over(Window.orderBy(desc('vech_wise_total_injury_death_count')))) \
            .filter(col('ordinality_injury_death_count').between(limit_start, limit_end)) \
            .withColumn("VEH_MAKE_ID_largest_injuries", col('VEH_MAKE_ID')) \
            .select('VEH_MAKE_ID_largest_injuries')


    except ValueError as ve:
        print(f"ValueError: {ve}")
        df_vehicle_wise_injury_death_count=spark.createDataFrame([(str(ve),)], ["issues_presists"])

    except Exception as e:
        print(f"Exception: {e}")
        df_vehicle_wise_injury_death_count=spark.createDataFrame([(str(e),)], ["issues_presists"])
    finally:
        return df_vehicle_wise_injury_death_count

get_top_vehicle_injuries(3,'5').show()

+----------------------------+
|VEH_MAKE_ID_largest_injuries|
+----------------------------+
|                      TOYOTA|
|                       DODGE|
|                      NISSAN|
+----------------------------+



In [ ]:
#Analysis 6: Which are the Top 3rd to 5th VEH_MAKE_IDs that contribute to a largest number of injuries including death

limiti_start=3
limiti_end=5
df_vechile_wise_injury_death_count=load_dataframe('Unit').select(['TOT_INJRY_CNT','DEATH_CNT','VEH_MAKE_ID'])\
.withColumn('total_injury_death_count',trim(col('TOT_INJRY_CNT'))+trim(col('DEATH_CNT')))\
.groupBy('VEH_MAKE_ID')\
.agg(sum(col('total_injury_death_count')).alias('vech_wise_total_injury_death_count'))
df_vechile_wise_injury_death_count.\
withColumn("ordinality_injury_death_count",rank().over(Window.orderBy(desc('vech_wise_total_injury_death_count'))))\
.filter(col('ordinality_injury_death_count')<=limiti_end).filter(col('ordinality_injury_death_count')>=limiti_start)\
.withColumn("VEH_MAKE_ID_largest_injures",col('VEH_MAKE_ID'))\
.select('VEH_MAKE_ID_largest_injures')\
.show(truncate=False)


+---------------------------+
|VEH_MAKE_ID_largest_injures|
+---------------------------+
|TOYOTA                     |
|DODGE                      |
|NISSAN                     |
+---------------------------+



In [ ]:
from pyspark.sql.functions import upper, col, count, sum,rank
from pyspark.sql.window import Window


#Analysis 6: Which are the Top 3rd to 5th VEH_MAKE_IDs that contribute to a largest number of injuries including death

df_vechile_wise_injury_death_count=load_dataframe('Unit').select(['TOT_INJRY_CNT','DEATH_CNT','VEH_MAKE_ID'])\
.withColumn('total_injury_death_count',trim(col('TOT_INJRY_CNT'))+trim(col('DEATH_CNT')))\
.groupBy('VEH_MAKE_ID')\
.agg(sum(col('total_injury_death_count')).alias('vech_wise_total_injury_death_count'))
df_vechile_wise_injury_death_count.\
withColumn("ordinality_injury_death_count",rank().over(Window.orderBy(desc('vech_wise_total_injury_death_count'))))\
.filter(col('ordinality_injury_death_count')<=5).filter(col('ordinality_injury_death_count')>=3).show(truncate=False)


+-----------+----------------------------------+-----------------------------+
|VEH_MAKE_ID|vech_wise_total_injury_death_count|ordinality_injury_death_count|
+-----------+----------------------------------+-----------------------------+
|TOYOTA     |4228.0                            |3                            |
|DODGE      |3146.0                            |4                            |
|NISSAN     |3118.0                            |5                            |
+-----------+----------------------------------+-----------------------------+



## Analysis 7: For all the body styles involved in crashes, mention the top ethnic user group of each unique body style

In [ ]:
def get_top_ethnic_groups_per_body_style() -> DataFrame:
    """
    Returns a DataFrame with the top ethnic user group for each unique body style involved in crashes.

    Returns:
        DataFrame: A DataFrame with columns 'VEH_BODY_STYL_ID' and 'PRSN_ETHNICITY_ID'.
    """
    try:
        # filter to exclude while selecting data in their groups
        common_na_filter = ['NA', 'OTHER', 'UNKNOWN', 'OTHER  (EXPLAIN IN NARRATIVE)']

        # filtering the ethnic details data
        ethinic_details_person = load_dataframe('PrimaryPerson').select(['PRSN_ETHNICITY_ID', 'CRASH_ID', 'UNIT_NBR']).filter(~col('PRSN_ETHNICITY_ID').isin(common_na_filter))

        # filtering the vehicle body style details data
        vechile_body_style = load_dataframe('Unit').select(['VEH_BODY_STYL_ID', 'CRASH_ID', 'UNIT_NBR']).filter(~col('VEH_BODY_STYL_ID').isin(common_na_filter))

        # ethnic groups per each vehicle body style
        vechile_body_style_ethinicity_counts = vechile_body_style.join(ethinic_details_person, ['CRASH_ID', 'UNIT_NBR'], 'inner') \
            .groupBy('VEH_BODY_STYL_ID', 'PRSN_ETHNICITY_ID') \
            .agg(count(col('PRSN_ETHNICITY_ID')).alias('count_of_ethnicity'))

        # rank window of the ethnic group for each vehicle body style
        window = Window.partitionBy('VEH_BODY_STYL_ID').orderBy(col("count_of_ethnicity").desc())

        # picking the top ethnic group
        vechile_wise_top_ethinic_category = vechile_body_style_ethinicity_counts.withColumn("ethnicity_rank", rank().over(window)) \
            .filter(col('ethnicity_rank') == 1) \
            .select(['VEH_BODY_STYL_ID', 'PRSN_ETHNICITY_ID'])


    except Exception as e:
        # Exception handling
        print(f"An error occurred: {str(e)}")
        vechile_wise_top_ethinic_category=spark.createDataFrame([(str(e),)], ["issues_presists"])
    finally:
        return vechile_wise_top_ethinic_category
get_top_ethnic_groups_per_body_style().show()

+--------------------+-----------------+
|    VEH_BODY_STYL_ID|PRSN_ETHNICITY_ID|
+--------------------+-----------------+
|           AMBULANCE|            WHITE|
|                 BUS|         HISPANIC|
|      FARM EQUIPMENT|            WHITE|
|          FIRE TRUCK|            WHITE|
|          MOTORCYCLE|            WHITE|
|NEV-NEIGHBORHOOD ...|            WHITE|
|        NOT REPORTED|            WHITE|
|PASSENGER CAR, 2-...|            WHITE|
|PASSENGER CAR, 4-...|            WHITE|
|              PICKUP|            WHITE|
|    POLICE CAR/TRUCK|            WHITE|
|   POLICE MOTORCYCLE|            WHITE|
|SPORT UTILITY VEH...|            WHITE|
|               TRUCK|            WHITE|
|       TRUCK TRACTOR|            WHITE|
|                 VAN|            WHITE|
|   YELLOW SCHOOL BUS|            BLACK|
+--------------------+-----------------+



In [ ]:
#Analysis 7: For all the body styles involved in crashes, mention the top ethnic user group of each unique body style
from pyspark.sql.functions import upper, col, count, sum,rank
from pyspark.sql.window import Window
from pyspark.sql.functions import count,desc

#filter to exclude while selecting data in their groups
common_na_filter=['NA','OTHER','UNKNOWN','OTHER  (EXPLAIN IN NARRATIVE)']

#filtering the ethinic details data
ethinic_details_person=load_dataframe('PrimaryPerson').select(['PRSN_ETHNICITY_ID','CRASH_ID','UNIT_NBR']).filter(~upper(col('PRSN_ETHNICITY_ID')).isin(common_na_filter))

ethinic_details_person.show(5,truncate=False)

#filtering the vechile body style details data
vechile_body_style=load_dataframe('Unit').select(['VEH_BODY_STYL_ID','CRASH_ID','UNIT_NBR']).filter(~upper(col('VEH_BODY_STYL_ID')).isin(common_na_filter))
vechile_body_style.show(5,truncate=False)

#ethinic groups per each vechile body style
vechile_body_style_ethinicity_counts=vechile_body_style.join(ethinic_details_person,['CRASH_ID','UNIT_NBR'],'inner')\
.groupBy('VEH_BODY_STYL_ID','PRSN_ETHNICITY_ID')\
.agg(count(col('PRSN_ETHNICITY_ID')).alias('count_of_ethnicity'))

#rank window of the ethinic group for each vechile body style
window = Window.partitionBy('VEH_BODY_STYL_ID',).orderBy(col("count_of_ethnicity").desc())

#picking the top ethinic group
vechile_body_style_ethinicity_counts.withColumn("ethnicity_rank", rank().over(window)).filter(col('ethnicity_rank')==1).show(truncate=False)


+-----------------+--------+--------+
|PRSN_ETHNICITY_ID|CRASH_ID|UNIT_NBR|
+-----------------+--------+--------+
|HISPANIC         |14768622|1       |
|WHITE            |14838637|1       |
|BLACK            |14838637|2       |
|WHITE            |14838641|1       |
|BLACK            |14838641|2       |
+-----------------+--------+--------+
only showing top 5 rows

+---------------------+--------+--------+
|VEH_BODY_STYL_ID     |CRASH_ID|UNIT_NBR|
+---------------------+--------+--------+
|PASSENGER CAR, 4-DOOR|14768622|1       |
|PASSENGER CAR, 4-DOOR|14838637|1       |
|TRUCK                |14838637|2       |
|TRUCK                |14838637|2       |
|PASSENGER CAR, 4-DOOR|14838641|1       |
+---------------------+--------+--------+
only showing top 5 rows

+---------------------------------+-----------------+------------------+--------------+
|VEH_BODY_STYL_ID                 |PRSN_ETHNICITY_ID|count_of_ethnicity|ethnicity_rank|
+---------------------------------+-----------------+-

## Analysis 8: Among the crashed cars, what are the Top 5 Zip Codes with highest number crashes with alcohols as the contributing factor to a crash (Use Driver Zip Code)


In [ ]:
# Analysis 8: Among the crashed cars, what are the Top 5 Zip Codes with highest
#number crashes with alcohols as the contributing factor to a crash (Use Driver Zip Code)
def get_top_zipcodes_with_alcohol_crashes()-> DataFrame:
    """
    Returns a DataFrame containing the top 5 zip codes with the highest number of crashes
    where alcohol is the contributing factor to the crash.
    """
    try:

        # Selecting crashed cars as body style
        crashed_cars_detail = load_dataframe('Unit') \
            .select(['CRASH_ID', 'UNIT_NBR', 'VEH_BODY_STYL_ID']) \
            .filter(trim(col('VEH_BODY_STYL_ID')).rlike('PASSENGER\\s*CAR'))

        # Selecting entries where alcohol consumption result is positive and its zip code
        crash_alcohol_zipcode_details = load_dataframe('PrimaryPerson') \
            .filter(col('PRSN_ALC_RSLT_ID') == 'Positive') \
            .filter(col('DRVR_ZIP') != 'NULL') \
            .select(['CRASH_ID', 'UNIT_NBR', 'PRSN_ALC_RSLT_ID', 'DRVR_ZIP'])

        # Rank window on the total number of crashes
        window = Window.orderBy(col("num_of_crash").desc())

        # Pulling up the crash zip code details and vehicle type and picking the top 5 zip codes with more crashes
        crash_alcohol_zipcode_zipcodes = crash_alcohol_zipcode_details.join(crashed_cars_detail, ['CRASH_ID', 'UNIT_NBR'], 'inner') \
            .groupBy('DRVR_ZIP') \
            .agg(count(col('CRASH_ID')).alias('num_of_crash')) \
            .withColumn("zip_crash_frequency_rank", row_number().over(window)) \
            .filter(col('zip_crash_frequency_rank') <= 5) \
            .select(['DRVR_ZIP'])


    except Exception as e:
        # Handle exceptions here
        print(f"An error occurred: {str(e)}")
        crash_alcohol_zipcode_zipcodes=spark.createDataFrame([(str(e),)], ["issues_presists"])
    finally:
        return crash_alcohol_zipcode_zipcodes

get_top_zipcodes_with_alcohol_crashes().show()

+--------+
|DRVR_ZIP|
+--------+
|   78521|
|   79936|
|   76010|
|   79938|
|   78240|
+--------+



In [ ]:
# Analysis 8: Among the crashed cars, what are the Top 5 Zip Codes with highest
#number crashes with alcohols as the contributing factor to a crash (Use Driver Zip Code)

from pyspark.sql.functions import row_number

#selecting car  as body style
crashed_cars_detail=load_dataframe('Unit') \
.select(['CRASH_ID','UNIT_NBR','VEH_BODY_STYL_ID'])\
.filter(trim(col('VEH_BODY_STYL_ID')).rlike('PASSENGER\\s*CAR') )
crashed_cars_detail.show(5,truncate=False)

#selecting the entires where alchol consumption result is positive and its zip code
crash_alchol_zipcode_details=load_dataframe('PrimaryPerson')\
.filter(col('PRSN_ALC_RSLT_ID')=='Positive')\
.filter(col('DRVR_ZIP')!='NULL')\
.select(['CRASH_ID','UNIT_NBR','PRSN_ALC_RSLT_ID','DRVR_ZIP'])
crash_alchol_zipcode_details.show(5,truncate=False)

#rank window on the order total number of crashes
window = Window.orderBy(col("num_of_crash").desc())


#pulling up the crash zip code details and vechile type and picking the top 5 zip codes with more crashes
crash_alchol_zipcode_details.join(crashed_cars_detail,['CRASH_ID','UNIT_NBR'],'inner')\
.groupBy('DRVR_ZIP')\
.agg(count(col('CRASH_ID')).alias('num_of_crash'))\
.withColumn("zip_crash_frequency_rank", row_number().over(window))\
.filter(col('zip_crash_frequency_rank')<=5)\
.select(['DRVR_ZIP'])\
.show(truncate=False)





+--------+--------+---------------------+
|CRASH_ID|UNIT_NBR|VEH_BODY_STYL_ID     |
+--------+--------+---------------------+
|14768622|1       |PASSENGER CAR, 4-DOOR|
|14838637|1       |PASSENGER CAR, 4-DOOR|
|14838641|1       |PASSENGER CAR, 4-DOOR|
|14838668|1       |PASSENGER CAR, 4-DOOR|
|14838669|2       |PASSENGER CAR, 4-DOOR|
+--------+--------+---------------------+
only showing top 5 rows

+--------+--------+----------------+--------+
|CRASH_ID|UNIT_NBR|PRSN_ALC_RSLT_ID|DRVR_ZIP|
+--------+--------+----------------+--------+
|14768622|1       |Positive        |77357   |
|14838637|1       |Positive        |13830   |
|14838668|1       |Positive        |78076   |
|14838669|2       |Positive        |76118   |
|14838670|1       |Positive        |75050   |
+--------+--------+----------------+--------+
only showing top 5 rows

+--------+
|DRVR_ZIP|
+--------+
|78521   |
|79936   |
|76010   |
|79938   |
|78240   |
+--------+



## Analysis 9: Count of Distinct Crash IDs where No Damaged Property was observed and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance

In [ ]:
# Analysis 9: Count of Distinct Crash IDs where No Damaged Property was observed
# and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance

def analyze_crash_data() -> DataFrame:
    """
    Analyzes crash data and returns a DataFrame.

    Returns:
        DataFrame: The analyzed crash data.
    """
    data = [Row(count_analyze_crash_data='not_able_to_convert')]
    try:
        # SELECTING CRASH ID where damaged property is present
        damage_crash_id = load_dataframe('Damages').select('CRASH_ID', 'DAMAGED_PROPERTY') \
            .filter(~upper(col('DAMAGED_PROPERTY')).rlike('NONE')).select('CRASH_ID').distinct()
        damage_crash_id = [i[0] for i in damage_crash_id.collect()]

        # Selecting insurance where it's availed
        insurance_on = load_dataframe('Charges').select(['CHARGE', 'CRASH_ID', 'UNIT_NBR']) \
            .filter(((upper(col('CHARGE')) == ('INSURANCE')) | (upper(col('CHARGE')).rlike('ON\\s*INSURANCE')) |
                     (upper(col('CHARGE')).rlike("DRIVER'S\\s*LICENSE\\s*INSURANCE"))) &
                    ((~upper(col('CHARGE')).rlike('FAIL')) & (~upper(col('CHARGE')).rlike('NO')) &
                     (~upper(col('CHARGE')).rlike('EXCLUDED')))).select(['CRASH_ID', 'UNIT_NBR'])

        # Crashes where damage level > 4 and selecting the crash_id and unit_nbr
        crshid_unitnbr_damage_level = load_dataframe('Unit').select(['CRASH_ID', 'UNIT_NBR', 'VEH_DMAG_SCL_1_ID',
                                                                     'VEH_DMAG_SCL_2_ID']) \
            .withColumn('EH_DMAG_SCL_damage_level1',
                        regexp_extract(col('VEH_DMAG_SCL_1_ID'), 'DAMAGED (\d)', 1).cast(IntegerType())) \
            .withColumn('EH_DMAG_SCL_damage_level2',
                        regexp_extract(col('VEH_DMAG_SCL_2_ID'), 'DAMAGED (\d)', 1).cast(IntegerType())) \
            .filter(col('EH_DMAG_SCL_damage_level1').isNotNull()) \
            .filter(col('EH_DMAG_SCL_damage_level2').isNotNull()) \
            .filter(col('EH_DMAG_SCL_damage_level1') > 4) \
            .filter(col('EH_DMAG_SCL_damage_level2') > 4) \
            .select(['CRASH_ID', 'UNIT_NBR'])

        # Joins of all the data like insurance, damage level, and insurance details
        temp_damage_level_filter = crshid_unitnbr_damage_level.filter(~col('CRASH_ID').isin(damage_crash_id))
        result = temp_damage_level_filter.join(insurance_on,
                                               ((temp_damage_level_filter['CRASH_ID'] != insurance_on['CRASH_ID']) &
                                                (temp_damage_level_filter['UNIT_NBR'] != insurance_on[
                                                    'UNIT_NBR'])), 'inner') \
            .select(temp_damage_level_filter['CRASH_ID'].alias('CRASH_ID_distinct')).distinct()
        data = [Row(count_analyze_crash_data=result.count())]


    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        return spark.createDataFrame(data)
analyze_crash_data().show()

+------------------------+
|count_analyze_crash_data|
+------------------------+
|                    1384|
+------------------------+



In [ ]:
#SELECTING CRASH ID where damage is there and neglecting the no damge crashes
no_damage_crash_id=load_dataframe('Damages').select('CRASH_ID','DAMAGED_PROPERTY')\
.filter(upper(col('DAMAGED_PROPERTY')).rlike('NONE')).select('DAMAGED_PROPERTY').distinct()
no_damage_crash_id.show(10,truncate=False)

+----------------+
|DAMAGED_PROPERTY|
+----------------+
|NONE1           |
|NONE            |
+----------------+



In [ ]:
# Analysis 9: Count of Distinct Crash IDs where No Damaged Property was observed
# and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance
# DAMAGED_PROPERTY
# null,none

from pyspark.sql.functions import length,regexp_extract
from pyspark.sql.types import IntegerType

#SELECTING CRASH ID where damage is there and neglecting the no damge crashes
# no_damage_crash_id=load_dataframe('Damages').select('CRASH_ID','DAMAGED_PROPERTY')\
# .filter(~upper(col('DAMAGED_PROPERTY')).rlike('NONE'))
damage_crash_id=load_dataframe('Damages').select('CRASH_ID','DAMAGED_PROPERTY')\
.filter(~upper(col('DAMAGED_PROPERTY')).rlike('NONE')).select('CRASH_ID').distinct()
# damage_crash_id.show(10,truncate=False)
damage_crash_id=[i[0]for i in damage_crash_id.collect()]

insurance_on=load_dataframe('Charges').select(['CHARGE','CRASH_ID','UNIT_NBR']).filter(((upper(col('CHARGE'))==('INSURANCE')) | (upper(col('CHARGE')).rlike('ON\\s*INSURANCE')) | \
 (upper(col('CHARGE')).rlike("DRIVER'S\\s*LICENSE\\s*INSURANCE")))&((~upper(col('CHARGE')).rlike('FAIL'))&(~upper(col('CHARGE')).rlike('NO'))&(~upper(col('CHARGE')).rlike('EXCLUDED'))))\
.select(['CRASH_ID','UNIT_NBR'])
# insurance_on.show(50,truncate=False)



crshid_unitnbr_damage_level=load_dataframe('Unit').select(['CRASH_ID','UNIT_NBR','VEH_DMAG_SCL_1_ID','VEH_DMAG_SCL_2_ID'])\
.withColumn('EH_DMAG_SCL_damage_level1',regexp_extract(col('VEH_DMAG_SCL_1_ID'), 'DAMAGED (\d)', 1).cast(IntegerType()))\
.withColumn('EH_DMAG_SCL_damage_level2',regexp_extract(col('VEH_DMAG_SCL_2_ID'), 'DAMAGED (\d)', 1).cast(IntegerType()))\
.filter(col('EH_DMAG_SCL_damage_level1').isNotNull())\
.filter(col('EH_DMAG_SCL_damage_level2').isNotNull())\
.filter(col('EH_DMAG_SCL_damage_level1')>4)\
.filter(col('EH_DMAG_SCL_damage_level2')>4)\
.select(['CRASH_ID','UNIT_NBR'])

# crshid_unitnbr_damage_level.orderBy('CRASH_ID').show(truncate=False)

temp_damage_level_filter=crshid_unitnbr_damage_level.filter(~col('CRASH_ID').isin(damage_crash_id))
temp_damage_level_filter.join(insurance_on,((temp_damage_level_filter['CRASH_ID']!=insurance_on['CRASH_ID'])\
                                            &(temp_damage_level_filter['UNIT_NBR']!=insurance_on['UNIT_NBR'])),'inner')\
.select(temp_damage_level_filter['CRASH_ID'].alias('CRASH_ID_distinct')).distinct().count()





# load_dataframe('Charges').select('CHARGE')\
# .filter((upper(col('CHARGE')).rlike('INSURANCE')) & (~upper(col('CHARGE')).rlike('NO'))&\
#                                                   (~upper(col('CHARGE')).rlike('FAIL'))\
#                                                   & (~upper(col('CHARGE')).rlike('WITHOUT'))& (~upper(col('CHARGE')).rlike('EXPIRED'))).distinct().show(50,truncate=False)

# load_dataframe('Charges').select(['CHARGE','CRASH_ID','UNIT_NBR']).filter(~((upper(col('CHARGE'))==('INSURANCE')) | (upper(col('CHARGE')).rlike('ON\\s*INSURANCE'))|(upper(col('CHARGE')).rlike('NO'))))\
# .select('CHARGE')\
# .distinct()\
# .show(50,truncate=False)
#
# |NO PROOF ON INSURANCE                   |
# |FMFR-EXCLUDED DRIVER ON INSURANCE POLICY|
# |FAIL TO SHOW PROOF ON INSURANCE         |


1384

In [ ]:
# Analysis 9: Count of Distinct Crash IDs where No Damaged Property was observed
# and Damage Level (VEH_DMAG_SCL~) is above 4 and car avails Insurance

from pyspark.sql.functions import length,regexp_extract
from pyspark.sql.types import IntegerType

#SELECTING CRASH ID where damage property is present
damage_crash_id=load_dataframe('Damages').select('CRASH_ID','DAMAGED_PROPERTY')\
.filter(~upper(col('DAMAGED_PROPERTY')).rlike('NONE')).select('CRASH_ID').distinct()

damage_crash_id=[i[0]for i in damage_crash_id.collect()]

#selecting insurance where its availed
insurance_on=load_dataframe('Charges').select(['CHARGE','CRASH_ID','UNIT_NBR']).filter(((upper(col('CHARGE'))==('INSURANCE')) | (upper(col('CHARGE')).rlike('ON\\s*INSURANCE')) | \
 (upper(col('CHARGE')).rlike("DRIVER'S\\s*LICENSE\\s*INSURANCE")))&((~upper(col('CHARGE')).rlike('FAIL'))&(~upper(col('CHARGE')).rlike('NO'))&(~upper(col('CHARGE')).rlike('EXCLUDED'))))\
.select(['CRASH_ID','UNIT_NBR'])


#crashes where damage level >4 and selecting the carsh_id and unit_nbr
crshid_unitnbr_damage_level=load_dataframe('Unit').select(['CRASH_ID','UNIT_NBR','VEH_DMAG_SCL_1_ID','VEH_DMAG_SCL_2_ID'])\
.withColumn('EH_DMAG_SCL_damage_level1',regexp_extract(col('VEH_DMAG_SCL_1_ID'), 'DAMAGED (\d)', 1).cast(IntegerType()))\
.withColumn('EH_DMAG_SCL_damage_level2',regexp_extract(col('VEH_DMAG_SCL_2_ID'), 'DAMAGED (\d)', 1).cast(IntegerType()))\
.filter(col('EH_DMAG_SCL_damage_level1').isNotNull())\
.filter(col('EH_DMAG_SCL_damage_level2').isNotNull())\
.filter(col('EH_DMAG_SCL_damage_level1')>4)\
.filter(col('EH_DMAG_SCL_damage_level2')>4)\
.select(['CRASH_ID','UNIT_NBR'])


#joins of all there data like insurance ,damage level and insurance details
temp_damage_level_filter=crshid_unitnbr_damage_level.filter(~col('CRASH_ID').isin(damage_crash_id))
temp_damage_level_filter.join(insurance_on,((temp_damage_level_filter['CRASH_ID']!=insurance_on['CRASH_ID'])\
                                            &(temp_damage_level_filter['UNIT_NBR']!=insurance_on['UNIT_NBR'])),'inner')\
.select(temp_damage_level_filter['CRASH_ID'].alias('CRASH_ID_distinct')).distinct().count()






1384

## Analysis 10: Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences, has licensed Drivers, used top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)

In [ ]:
def get_top_5_vehicle_makes() -> DataFrame:
    """
    Determine the Top 5 Vehicle Makes where drivers are charged with speeding related offences,
    has licensed Drivers, used top 10 used vehicle colours and has car licensed with the Top 25 states with highest number of offences.

    Returns:
    - DataFrame: Top 5 Vehicle Makes with the given conditions.
    """
    try:
        # top 10 used vehicle colors
        top_10_used_vehicle_colours = load_dataframe('Unit').filter(col('VEH_COLOR_ID') != 'NA').groupBy('VEH_COLOR_ID').agg(count(col('VEH_COLOR_ID')).alias('count_of_color')).withColumn('color_rank', rank().over(Window.orderBy(desc('count_of_color'))))\
            .filter(col('color_rank') <= 10).select('VEH_COLOR_ID')
        top_10_colours = [i[0] for i in top_10_used_vehicle_colours.collect()]

        # car licensed with the Top 25 states with highest number of offenses
        top_25_offence_states = load_dataframe('Unit').filter(col('VEH_LIC_STATE_ID') != 'NA').groupBy('VEH_LIC_STATE_ID').agg(count(col('VEH_LIC_STATE_ID')).alias('count_of_offence'))\
            .withColumn('offence_rank', rank().over(Window.orderBy(desc('count_of_offence'))))\
            .filter(col('offence_rank') <= 25).select('VEH_LIC_STATE_ID')
        top_25_offensive_states = [i[0] for i in top_25_offence_states.collect()]

        # records filtering of top colors and top_25 offensive states
        vech_top_colors_offenstates = load_dataframe('Unit').select('CRASH_ID', 'UNIT_NBR', 'VEH_MAKE_ID', 'VEH_COLOR_ID')\
            .filter(col('VEH_COLOR_ID').isin(top_10_colours)).\
            filter(col('VEH_LIC_STATE_ID').isin(top_25_offensive_states)).select('CRASH_ID', 'UNIT_NBR', 'VEH_MAKE_ID')

        # drivers are charged with speeding related offenses
        speed_related_charges = load_dataframe('Charges').filter(lower(col('CHARGE')).rlike('speed')).select('CRASH_ID', 'UNIT_NBR')

        # licensed Drivers
        liscensed_members = load_dataframe('PrimaryPerson').filter((col('DRVR_LIC_STATE_ID') != 'NA') & (col('DRVR_LIC_STATE_ID') != 'UNLICENSED')).select('CRASH_ID', 'UNIT_NBR')

        # top 5 makes with given condition
        vechile_make_final_rank = vech_top_colors_offenstates.join(speed_related_charges, ['CRASH_ID', 'UNIT_NBR'], 'inner')\
            .join(liscensed_members, ['CRASH_ID', 'UNIT_NBR'], 'inner')\
            .groupBy('VEH_MAKE_ID')\
            .agg(count(col('VEH_MAKE_ID')).alias('count_of_vech_make'))\
            .withColumn('vech_make_rank', rank().over(Window.orderBy(desc('count_of_vech_make'))))\
            .filter(col('vech_make_rank') <= 5).select('VEH_MAKE_ID')

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        vechile_make_final_rank=spark.createDataFrame([(str(e),)], ["issues_presists"])
    finally:
        return vechile_make_final_rank
get_top_5_vehicle_makes().show()

+-----------+
|VEH_MAKE_ID|
+-----------+
|       FORD|
|  CHEVROLET|
|     TOYOTA|
|      DODGE|
|      HONDA|
+-----------+



In [ ]:
# 10.	Analysis 10: Determine the Top 5 Vehicle Makes
# where drivers are charged with speeding related offences,
# has licensed Drivers, used top 10 used vehicle colours
# and has car licensed with the Top 25 states with highest number of offences (to be deduced from the data)
from pyspark.sql.functions import desc

#  top 10 used vehicle colours
top_10_used_vehicle_colours=load_dataframe('Unit').filter(col('VEH_COLOR_ID')!='NA').groupBy('VEH_COLOR_ID').agg(count(col('VEH_COLOR_ID')).alias('count_of_color')).withColumn('color_rank',rank().over(Window.orderBy(desc('count_of_color'))))\
.filter(col('color_rank')<=10).select('VEH_COLOR_ID')
top_10_colours=[i[0]for i in top_10_used_vehicle_colours.collect()]

# car licensed with the Top 25 states with highest number of offences
top_25_offence_states=load_dataframe('Unit').filter(col('VEH_LIC_STATE_ID')!='NA').groupBy('VEH_LIC_STATE_ID').agg(count(col('VEH_LIC_STATE_ID')).alias('count_of_offence'))\
.withColumn('offence_rank',rank().over(Window.orderBy(desc('count_of_offence'))))\
.filter(col('offence_rank')<=25).select('VEH_LIC_STATE_ID')
top_25_offensive_states=[i[0]for i in top_25_offence_states.collect()]

#records filtering  of top colors and top_25 offensive states
vech_top_colors_offenstates=load_dataframe('Unit').select('CRASH_ID','UNIT_NBR','VEH_MAKE_ID','VEH_COLOR_ID')\
.filter(col('VEH_COLOR_ID').isin(top_10_colours)).\
filter(col('VEH_LIC_STATE_ID').isin(top_25_offensive_states)).select('CRASH_ID','UNIT_NBR','VEH_MAKE_ID')

# drivers are charged with speeding related offences
speed_related_charges=load_dataframe('Charges').filter(lower(col('CHARGE')).rlike('speed')).select('CRASH_ID','UNIT_NBR')
#licensed Drivers
liscensed_members=load_dataframe('PrimaryPerson').filter((col('DRVR_LIC_STATE_ID')!='NA') & (col('DRVR_LIC_STATE_ID')!='UNLICENSED')).select('CRASH_ID','UNIT_NBR')

#top 5 makes with given condition
vechile_make_final_rank=vech_top_colors_offenstates.join(speed_related_charges,['CRASH_ID','UNIT_NBR'],'inner')\
.join(liscensed_members,['CRASH_ID','UNIT_NBR'],'inner')\
.groupBy('VEH_MAKE_ID')\
.agg(count(col('VEH_MAKE_ID')).alias('count_of_vech_make'))\
.withColumn('vech_make_rank',rank().over(Window.orderBy(desc('count_of_vech_make'))))\
.filter(col('vech_make_rank')<=5).select('VEH_MAKE_ID')
vechile_make_final_rank.show(truncate=False)




+-----------+
|VEH_MAKE_ID|
+-----------+
|FORD       |
|CHEVROLET  |
|TOYOTA     |
|DODGE      |
|HONDA      |
+-----------+



In [ ]:
for i in datasets_config.keys():
  print(i)
  load_dataframe(i).show(5)

Charges
+--------+--------+--------+--------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|              CHARGE|CITATION_NBR|
+--------+--------+--------+--------------------+------------+
|14768622|       1|       1|DRIVING WHILE INT...|        NULL|
|14838637|       1|       1|                 DWI|  1600000015|
|14838641|       1|       1|RAN RED LIGHT SOL...|      L20440|
|14838641|       2|       1|NO DRIVER'S LICEN...|      L23141|
|14838668|       1|       1|DRIVING WHILE INT...|TX4IC50SRJD3|
+--------+--------+--------+--------------------+------------+
only showing top 5 rows

Endorsements
+--------+--------+------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_ENDORS_ID|
+--------+--------+------------------+
|14768622|       1|              NONE|
|14838637|       1|              NONE|
|14838637|       2|              NONE|
|14838641|       1|              NONE|
|14838641|       2|        UNLICENSED|
+--------+--------+------------------+
only showing top 5 rows

Restrict
+-

In [ ]:
for i in datasets_config.keys():
  print(i)
  load_dataframe(i).show(5)

Charges
+--------+--------+--------+--------------------+------------+
|CRASH_ID|UNIT_NBR|PRSN_NBR|              CHARGE|CITATION_NBR|
+--------+--------+--------+--------------------+------------+
|14768622|       1|       1|DRIVING WHILE INT...|        NULL|
|14838637|       1|       1|                 DWI|  1600000015|
|14838641|       1|       1|RAN RED LIGHT SOL...|      L20440|
|14838641|       2|       1|NO DRIVER'S LICEN...|      L23141|
|14838668|       1|       1|DRIVING WHILE INT...|TX4IC50SRJD3|
+--------+--------+--------+--------------------+------------+
only showing top 5 rows

Endorsements
+--------+--------+------------------+
|CRASH_ID|UNIT_NBR|DRVR_LIC_ENDORS_ID|
+--------+--------+------------------+
|14768622|       1|              NONE|
|14838637|       1|              NONE|
|14838637|       2|              NONE|
|14838641|       1|              NONE|
|14838641|       2|        UNLICENSED|
+--------+--------+------------------+
only showing top 5 rows

Restrict
+-